In [45]:
import arcpy
import requests
import os
import psycopg2
import random
from pathlib import Path

In [46]:
# Set workspace
os.chdir(r'E:\ArcGIS_2\Lab3')
wksp = os.getcwd()

## 1. Data preparation

In [47]:
# Retrieve temperature data from PostGIS database
arcpy.management.MakeQueryLayer(
    input_database=os.path.join(wksp, ""),
    out_layer_name="temp_stations",
    query="SELECT id, min_tmpf, geom FROM stations WHERE date = '2023-03'",
    oid_fields="id",
    shape_type="POINT",
    srid="4326",
    spatial_reference='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;0 1;0 1;8.98315284119521E-09;2;2;IsHighPrecision',
    spatial_properties="DO_NOT_DEFINE_SPATIAL_PROPERTIES",
    m_values="DO_NOT_INCLUDE_M_VALUES",
    z_values="DO_NOT_INCLUDE_Z_VALUES",
    extent='-98.0690216979786 43.2052294998382 -88.6618510633838 49.6779752981444 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'
)

<Result 'temp_stations'>

In [48]:
# Create a copy of the temperature as a shapefile in the workspace
arcpy.management.CopyFeatures(
    in_features="temp_stations",
    out_feature_class=os.path.join(wksp, "temp_stations.shp"),
    config_keyword="",
    spatial_grid_1=None,
    spatial_grid_2=None,
    spatial_grid_3=None
)

<Result 'E:\\ArcGIS_2\\Lab3\\temp_stations.shp'>

In [49]:
# Set the path to input temperature, and output training and validation shapefiles
input_shapefile = "temp_stations.shp"
training_shapefile = os.path.join(wksp, "training_shapefile.shp")
validation_shapefile = os.path.join(wksp, "validation_shapefile.shp")

# Set the percentage of features to use for training
training_percent = 70

# Create a list of ObjectIDs for all features in the input shapefile
all_ids = [row[0] for row in arcpy.da.SearchCursor(input_shapefile, ["OID@"])]

# Calculate the number of features to use for training
num_training = int((len(all_ids) * training_percent) / 100)

# Randomly select the ObjectIDs for the training features
training_ids = random.sample(all_ids, num_training)

# Create separate lists of ObjectIDs for the validation and training features
validation_ids = [id for id in all_ids if id not in training_ids]

In [50]:
# Use the selected ObjectIDs to create new shapefiles for 
# i) training 
training = arcpy.management.SelectLayerByAttribute(input_shapefile, "NEW_SELECTION", "FID IN {}".format(tuple(training_ids)))
arcpy.management.CopyFeatures(training, training_shapefile)

# ii) validation 
validation = arcpy.management.SelectLayerByAttribute(input_shapefile, "NEW_SELECTION", "FID IN {}".format(tuple(validation_ids)))
arcpy.management.CopyFeatures(validation, validation_shapefile)

<Result 'E:\\ArcGIS_2\\Lab3\\validation_shapefile.shp'>

## 2. Interpolation

In [54]:
# Interpolate the temperature using 3 methods: IDW, Kriging, and GPI

arcpy.ddd.Idw(
    in_point_features="training_shapefile.shp",
    z_field="min_tmpf",
    out_raster=os.path.join(wksp, "IDW.tif"),
    cell_size=0.1,
    power=2,
    search_radius="VARIABLE 12",
    in_barrier_polyline_features=None
)

arcpy.ddd.Kriging(
    in_point_features="training_shapefile.shp",
    z_field="min_tmpf",
    out_surface_raster=os.path.join(wksp, "Kriging.tif"),
    semiVariogram_props="Spherical 0.021245 # # #",
    cell_size=0.1,
    search_radius="VARIABLE 12",
    out_variance_prediction_raster=None
)

arcpy.ga.GlobalPolynomialInterpolation(
    in_features="training_shapefile.shp",
    z_field="min_tmpf",
    out_ga_layer=None,
    out_raster=os.path.join(wksp, "GPI.tif"),
    cell_size=0.1,
    power=1,
    weight_field=None
)

<Result ''>

## 3. Accuracy assessment

In [55]:
def accuracy_assessment (raster, validation_data):
    """
    Calculate the RMSE of the interpolations by comparing the values to the validation data
    
    Input:
    - raster: interpolation method
    - validation_data: ground truth 
    
    """
        
    # Output name and path of the shapefile comparing ground truth vs classified
    output_acc = 'Acc_' + Path(raster).stem + '.shp'
    acc_table = os.path.join(wksp, output_acc)
    
    # Output name and path of the table that saves the RMSE for each interpolation
    output_stat = 'Acc_' + Path(raster).stem + '_stat.dbf'
    stat_table = os.path.join(wksp, output_stat)
    
    # Extract the predicted values and save them to the validation data's attribute table
    arcpy.sa.ExtractValuesToPoints(
        in_point_features=validation_data,
        in_raster=raster,
        out_point_features=acc_table,
        interpolate_values="NONE",
        add_attributes="ALL"
    )
    
    # Rename the default fields 
    arcpy.management.CalculateField(
        in_table=acc_table,
        field="GrndTruth",
        expression="!min_tmpf!",
        expression_type="PYTHON3",
        code_block="",
        field_type="FLOAT",
        enforce_domains="NO_ENFORCE_DOMAINS"
    )
    arcpy.management.CalculateField(
        in_table=acc_table,
        field="Classified",
        expression="!RASTERVALU!",
        expression_type="PYTHON3",
        code_block="",
        field_type="FLOAT",
        enforce_domains="NO_ENFORCE_DOMAINS"
    )
    arcpy.management.DeleteField(
        in_table=acc_table,
        drop_field="min_tmpf;RASTERVALU",
        method="DELETE_FIELDS"
    )
    
    # Calculate the squared error
    arcpy.management.CalculateField(
        in_table=acc_table,
        field="Sq_error",
        expression="math.pow(!GrndTruth! - !Classified!, 2)",
        expression_type="PYTHON3",
        code_block="",
        field_type="FLOAT",
        enforce_domains="NO_ENFORCE_DOMAINS"
    )
    
    # Create a new statistic table and calculate RMSE
    arcpy.analysis.Statistics(
        in_table=acc_table,
        out_table=stat_table,
        statistics_fields="Sq_error SUM",
        case_field=None,
        concatenation_separator=""
    )    
    arcpy.management.CalculateField(
        in_table=stat_table,
        field="RMSE",
        expression="math.sqrt(!SUM_Sq_err! / !FREQUENCY!)",
        expression_type="PYTHON3",
        code_block="",
        field_type="FLOAT",
        enforce_domains="NO_ENFORCE_DOMAINS"
    )

In [56]:
# Create lists with the raster names of the interpolations with and without extension
interpolations = ['IDW.tif', 'Kriging.tif', 'GPI.tif']
interpolators  = ['IDW', 'Kriging', 'GPI']

# Run the accuracy assesment for each interpolation
for i in range(len(interpolations)):
    accuracy_assessment(interpolations[i], "validation_shapefile.shp")

# Merge the accuracy tables     
arcpy.management.Merge(
    inputs="Acc_IDW_stat.dbf;Acc_Kriging_stat.dbf;Acc_GPI_stat.dbf",
    output="Accuracy_assessment.dbf",
    field_mappings='Interpolat "Interpolat" true true false 255 Text 0 0,First,#;FREQUENCY "FREQUENCY" true true false 10 Long 0 10,First,#,Acc_IDW_stat,FREQUENCY,-1,-1,Acc_Kriging_stat,FREQUENCY,-1,-1,Acc_GPI_stat,FREQUENCY,-1,-1;SUM_Sq_err "SUM_Sq_err" true true false 19 Double 0 0,First,#,Acc_IDW_stat,SUM_Sq_err,-1,-1,Acc_Kriging_stat,SUM_Sq_err,-1,-1,Acc_GPI_stat,SUM_Sq_err,-1,-1;RMSE "RMSE" true true false 13 Float 0 0,First,#,Acc_IDW_stat,RMSE,-1,-1,Acc_Kriging_stat,RMSE,-1,-1,Acc_GPI_stat,RMSE,-1,-1',
    add_source="NO_SOURCE_INFO"
)

# Update the merged table with the name of each interpolator
with arcpy.da.UpdateCursor("Accuracy_assessment.dbf", ['Interpolat']) as cursor:
    for i, row in enumerate(cursor):
        if i < len(interpolators):
            row[0] = interpolators[i]
        else:
            break
        cursor.updateRow(row)

# Delete the cursor to release locks on the data
del cursor

In [57]:
# Find the interpolator with the lowest RMSE
methods = {}
fields = ["Interpolat", "RMSE"]
with arcpy.da.SearchCursor('Accuracy_assessment.dbf', fields) as cursor:
    for row in cursor:
        methods[row[0]] = row[1]

best_interpolator = min(methods, key=methods.get)

In [58]:
# Clip interpolation to MN borders 
output_clip = os.path.join(wksp, best_interpolator + '_mn.tif')
out_raster = arcpy.sa.ExtractByMask(
    in_raster=best_interpolator+'.tif',
    in_mask_data="minnesota.shp",
    extraction_area="INSIDE",
    analysis_extent='-97.239102895829 43.499445217943 -89.6516983029999 49.0583312990001 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'
)
out_raster.save(output_clip)

# Convert raster to point shapefile
output_point_shp = os.path.join(wksp, best_interpolator + '.shp')
arcpy.conversion.RasterToPoint(
    in_raster=output_clip,
    out_point_features=output_point_shp,
    raster_field="Value"
)

<Result 'E:\\ArcGIS_2\\Lab3\\Kriging.shp'>

## 4. Save to PostGIS database

In [59]:
# Connect to PostGIS database
connection = psycopg2.connect(host = '',
                              port = '',
                              database = '',
                              user = '',
                              password = '',
                             )

### 4.1. Accuracy assessment table

In [60]:
# Path and fields of the data to load to the database
data = os.path.join(wksp, "Accuracy_assessment.dbf")
fields = ["OID", "Interpolat", "FREQUENCY", "SUM_Sq_err", "RMSE"]

# Create SQL table
cursor = connection.cursor()
cursor.execute("DROP TABLE IF EXISTS accuracy_assessment")
cursor.execute("""
    CREATE TABLE accuracy_assessment (
        OID INT,
        Interpolat VARCHAR,
        FREQUENCY INT,
        SUM_Sq_err DOUBLE PRECISION,
        RMSE DOUBLE PRECISION)
""")

# Populate table
with arcpy.da.SearchCursor(data, fields) as da_cursor:
    for row in da_cursor:
        cursor.execute("INSERT INTO accuracy_assessment (OID, Interpolat, FREQUENCY, SUM_Sq_err, RMSE) VALUES (%s, %s, %s, %s, %s)", (row[0], row[1], row[2], row[3], row[4]))

connection.commit()

### 4.2. Best interpolation

In [61]:
# Create table name with the name of the best interpolator
point_table = best_interpolator.lower()

fields = ["pointid", "grid_code", "Shape@WKT"]

# Create SQL table
cursor = connection.cursor()
cursor.execute(f"DROP TABLE IF EXISTS {point_table}")
cursor.execute(f"""
    CREATE TABLE {point_table} (
        pointid INT,
        grid_code DOUBLE PRECISION)
""")

cursor.execute(f"""
    SELECT AddGeometryColumn('{point_table}', 'geom', 4326, 'POINT', 2)
""")

# Populate table
with arcpy.da.SearchCursor(output_point_shp, fields) as da_cursor:
    for row in da_cursor:
        wkt = row[2]
        cursor.execute(f"INSERT INTO {point_table} (pointid, grid_code, geom) VALUES (%s, %s, ST_GeomFromText(%s, 4326))", (row[0], row[1], wkt))

connection.commit()

### 4.3. Error table for best interpolation

In [62]:
# Path of the shapefile
data = os.path.join(wksp, 'Acc_' + best_interpolator + '.shp')

# Create the table name to use in PostGIS database
table_name = best_interpolator.lower() + '_error_estimation'

fields = ["GrndTruth", "Classified", "Sq_error", "Shape@WKT"]

# Create SQL table
cursor = connection.cursor()
cursor.execute(f"DROP TABLE IF EXISTS {table_name}")
cursor.execute(f"""
    CREATE TABLE {table_name} (
        GrndTruth DOUBLE PRECISION,
        Classified DOUBLE PRECISION,
        Sq_error DOUBLE PRECISION)
""")

cursor.execute(f"""
    SELECT AddGeometryColumn('{table_name}', 'geom', 4326, 'POINT', 2)
""")

# Populate table
with arcpy.da.SearchCursor(data, fields) as da_cursor:
    for row in da_cursor:
        wkt = row[3]
        cursor.execute(f"INSERT INTO {table_name} (GrndTruth, Classified, Sq_error, geom) VALUES (%s, %s, %s, ST_GeomFromText(%s, 4326))", (row[0], row[1], row[2], wkt))

connection.commit()

In [63]:
# Close database connection
connection.close()